# JIT Compilation with BrainState

Just-In-Time (JIT) compilation is one of JAX's most powerful features, allowing Python code to run at near-C speeds. BrainState provides state-aware JIT compilation that works seamlessly with stateful computations.

In this tutorial, you will learn:

- ⚡ **JIT Basics** - Speed up your code with `@jit`
- 🔧 **Static vs Dynamic** - Understanding argument types
- 🎯 **Optimization Techniques** - Best practices for JIT
- 📊 **Performance Analysis** - Measuring speedups
- 🧠 **Stateful JIT** - Compiling with BrainState objects

## Why JIT?

**Benefits:**
- ⚡ **10-100x speedup** for numerical computations
- 🚀 **GPU/TPU acceleration** automatic
- 🔄 **Optimized execution** - fused operations, reduced memory
- 📈 **Scales to large models** efficiently

In [ ]:
import brainstate as bst
import jax
import jax.numpy as jnp
import time
import matplotlib.pyplot as plt
import numpy as np

## 1. JIT Basics

The simplest way to use JIT is with the `@jit` decorator:

### Pure Functions

In [ ]:
# Without JIT
def compute_sum_no_jit(x):
    """Sum of squares without JIT."""
    return jnp.sum(x ** 2)

# With JIT
@bst.transform.jit
def compute_sum_jit(x):
    """Sum of squares with JIT."""
    return jnp.sum(x ** 2)

# Test data
x = bst.random.randn(1000)

# First call (compilation + execution)
print("First call (includes compilation):")
start = time.time()
result_jit = compute_sum_jit(x)
first_time = time.time() - start
print(f"  JIT result: {result_jit:.4f}, Time: {first_time*1000:.3f} ms")

# Second call (cached, just execution)
print("\nSecond call (cached):")
start = time.time()
result_jit = compute_sum_jit(x)
jit_time = time.time() - start
print(f"  JIT result: {result_jit:.4f}, Time: {jit_time*1000:.3f} ms")

# No JIT
start = time.time()
result_no_jit = compute_sum_no_jit(x)
no_jit_time = time.time() - start
print(f"  No JIT result: {result_no_jit:.4f}, Time: {no_jit_time*1000:.3f} ms")

print(f"\n✅ Speedup: {no_jit_time / jit_time:.1f}x")

### Understanding Compilation

JIT compilation happens in two stages:

1. **First call**: Trace → Optimize → Compile → Execute
2. **Subsequent calls**: Execute (cached)

Let's visualize this:

In [ ]:
@bst.transform.jit
def complex_computation(x, n_iterations=100):
    """More complex computation for timing."""
    result = x
    for i in range(n_iterations):
        result = jnp.sin(result) + jnp.cos(result)
        result = result / (jnp.sum(result) + 1e-8)
    return result

# Measure compilation + execution
x = bst.random.randn(100)

times = []
for i in range(5):
    start = time.time()
    result = complex_computation(x)
    elapsed = (time.time() - start) * 1000
    times.append(elapsed)
    print(f"Call {i+1}: {elapsed:.2f} ms")

# Plot
plt.figure(figsize=(10, 5))
plt.bar(range(1, 6), times, color=['red', 'green', 'green', 'green', 'green'], alpha=0.7)
plt.xlabel('Call Number')
plt.ylabel('Time (ms)')
plt.title('JIT Compilation: First Call vs Cached Calls', fontweight='bold')
plt.axhline(y=times[1], color='green', linestyle='--', alpha=0.5, label='Cached time')
plt.legend(['Cached baseline', 'Execution time'])
plt.grid(alpha=0.3)
plt.show()

print(f"\nFirst call (compilation): {times[0]:.2f} ms")
print(f"Cached calls (average): {np.mean(times[1:]):.2f} ms")
print(f"Compilation overhead: {times[0] - times[1]:.2f} ms")

## 2. Static vs Dynamic Arguments

JAX distinguishes between **static** (compile-time) and **dynamic** (runtime) arguments.

### Dynamic Arguments (Default)

Most arguments are dynamic - the function reuses the compiled code:

In [ ]:
@bst.transform.jit
def add_arrays(x, y):
    return x + y

# Different array values, same shape - reuses compilation
a = jnp.array([1.0, 2.0, 3.0])
b = jnp.array([4.0, 5.0, 6.0])
c = jnp.array([7.0, 8.0, 9.0])
d = jnp.array([10.0, 11.0, 12.0])

print("Calling with different values, same shape:")
print(f"add_arrays(a, b) = {add_arrays(a, b)}  # Compiles")
print(f"add_arrays(c, d) = {add_arrays(c, d)}  # Reuses compilation")

# Different shape - triggers recompilation
e = jnp.array([1.0, 2.0])
f = jnp.array([3.0, 4.0])

print(f"\nadd_arrays(e, f) = {add_arrays(e, f)}  # Recompiles (different shape)")

print("\n✅ Array values are dynamic (no recompilation)")
print("⚠️  Array shapes are static (triggers recompilation)")

### Static Arguments

Use `static_argnums` to mark arguments as static:

In [ ]:
@bst.transform.jit(static_argnums=(1,))  # n is static
def power(x, n):
    """Raise x to the power n."""
    result = x
    for _ in range(n - 1):
        result = result * x
    return result

x = jnp.array([2.0, 3.0])

print("Static argument (n):")
print(f"power(x, 2) = {power(x, 2)}  # Compiles for n=2")
print(f"power(x, 2) = {power(x, 2)}  # Reuses")
print(f"power(x, 3) = {power(x, 3)}  # Recompiles for n=3")
print(f"power(x, 3) = {power(x, 3)}  # Reuses")

print("\n⚠️  Each unique static value triggers a separate compilation")
print("✅ Use static args for: loop bounds, shape parameters, flags")

### Practical Example: Configurable Network Layer

In [ ]:
class ConfigurableLayer(bst.graph.Node):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.w = bst.ParamState(bst.random.randn(in_features, out_features) * 0.1)
        self.b = bst.ParamState(jnp.zeros(out_features))
    
    @bst.transform.jit(static_argnums=(2,))  # use_activation is static
    def forward(self, x, use_activation=True):
        """Forward with optional activation."""
        out = x @ self.w.value + self.b.value
        if use_activation:
            out = jnp.maximum(0, out)  # ReLU
        return out

# Create layer
bst.random.seed(42)
layer = ConfigurableLayer(10, 5)

x = bst.random.randn(10)

print("With activation:")
y1 = layer.forward(x, use_activation=True)
print(f"  Output: {y1}")

print("\nWithout activation:")
y2 = layer.forward(x, use_activation=False)
print(f"  Output: {y2}")

print("\n✅ Two separate compiled versions (use_activation=True/False)")

## 3. JIT with Stateful Objects

BrainState allows JIT compilation of stateful computations:

### Stateful Counter

In [ ]:
class Counter(bst.graph.Node):
    def __init__(self):
        super().__init__()
        self.count = bst.ShortTermState(jnp.array(0))
    
    @bst.transform.jit
    def increment(self, value):
        """Increment counter by value."""
        self.count.value = self.count.value + value
        return self.count.value

# Create and use
counter = Counter()

print("Stateful JIT compilation:")
for i in range(5):
    result = counter.increment(jnp.array(i + 1))
    print(f"  Increment by {i+1}: count = {result}")

print("\n✅ State updates work inside JIT-compiled functions")
print("✅ BrainState handles state threading automatically")

### Neural Network with JIT

In [ ]:
class MLPWithJIT(bst.graph.Node):
    def __init__(self, layers):
        super().__init__()
        self.linears = []
        for i in range(len(layers) - 1):
            linear = bst.nn.Linear(layers[i], layers[i+1])
            setattr(self, f'linear{i}', linear)
            self.linears.append(linear)
    
    @bst.transform.jit
    def __call__(self, x):
        """JIT-compiled forward pass."""
        for i, linear in enumerate(self.linears):
            x = linear(x)
            if i < len(self.linears) - 1:  # No activation on last layer
                x = jnp.maximum(0, x)  # ReLU
        return x

# Create network
bst.random.seed(0)
model = MLPWithJIT([784, 256, 128, 10])

# Benchmark
x = bst.random.randn(784)

# Warmup (compilation)
_ = model(x)

# Measure
n_runs = 1000
start = time.time()
for _ in range(n_runs):
    y = model(x)
elapsed = (time.time() - start) * 1000 / n_runs

print(f"MLP with JIT:")
print(f"  Input: {x.shape}")
print(f"  Output: {y.shape}")
print(f"  Average time: {elapsed:.4f} ms/forward")
print(f"  Throughput: {1000/elapsed:.0f} forwards/second")

## 4. Performance Analysis

Let's compare JIT vs non-JIT performance across different scenarios:

### Matrix Multiplication Benchmark

In [ ]:
def matmul_no_jit(A, B):
    return jnp.dot(A, B)

@bst.transform.jit
def matmul_jit(A, B):
    return jnp.dot(A, B)

# Test different sizes
sizes = [10, 50, 100, 500, 1000]
times_no_jit = []
times_jit = []

for size in sizes:
    A = bst.random.randn(size, size)
    B = bst.random.randn(size, size)
    
    # Warmup JIT
    _ = matmul_jit(A, B)
    
    # No JIT
    start = time.time()
    for _ in range(10):
        _ = matmul_no_jit(A, B)
    time_no_jit = (time.time() - start) / 10
    times_no_jit.append(time_no_jit * 1000)
    
    # JIT
    start = time.time()
    for _ in range(10):
        _ = matmul_jit(A, B)
    time_jit = (time.time() - start) / 10
    times_jit.append(time_jit * 1000)

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sizes, times_no_jit, 'o-', label='No JIT', linewidth=2, markersize=8)
plt.plot(sizes, times_jit, 's-', label='JIT', linewidth=2, markersize=8)
plt.xlabel('Matrix Size')
plt.ylabel('Time (ms)')
plt.title('Execution Time: JIT vs No JIT', fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.yscale('log')

plt.subplot(1, 2, 2)
speedups = [no_jit / jit for no_jit, jit in zip(times_no_jit, times_jit)]
plt.bar(range(len(sizes)), speedups, color='green', alpha=0.7)
plt.xticks(range(len(sizes)), sizes)
plt.xlabel('Matrix Size')
plt.ylabel('Speedup (x)')
plt.title('JIT Speedup Factor', fontweight='bold')
plt.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='No speedup')
plt.legend()
plt.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nSpeedup Summary:")
for size, speedup in zip(sizes, speedups):
    print(f"  Size {size:4d}: {speedup:.2f}x faster")

## 5. Optimization Tips and Best Practices

### Tip 1: Minimize Python Overhead

In [ ]:
# ❌ BAD: Python loop (not JIT-friendly)
def sum_bad(arr):
    total = 0
    for x in arr:
        total += x
    return total

# ✅ GOOD: JAX operations
@bst.transform.jit
def sum_good(arr):
    return jnp.sum(arr)

arr = bst.random.randn(1000)

# Compare
start = time.time()
for _ in range(100):
    _ = sum_bad(arr)
time_bad = (time.time() - start) * 1000 / 100

_ = sum_good(arr)  # Warmup
start = time.time()
for _ in range(100):
    _ = sum_good(arr)
time_good = (time.time() - start) * 1000 / 100

print(f"Python loop: {time_bad:.4f} ms")
print(f"JAX operations: {time_good:.4f} ms")
print(f"Speedup: {time_bad / time_good:.1f}x")
print("\n✅ Use JAX/NumPy operations instead of Python loops")

### Tip 2: Avoid Side Effects

In [ ]:
# ❌ BAD: Side effects (modifying external state)
global_list = []

def bad_function(x):
    global_list.append(x)  # Side effect!
    return x * 2

# ✅ GOOD: Pure function or BrainState states
@bst.transform.jit
def good_function(x):
    return x * 2

# With BrainState states (OK)
class GoodCounter(bst.graph.Node):
    def __init__(self):
        super().__init__()
        self.values = bst.ShortTermState(jnp.array([]))
    
    @bst.transform.jit
    def append(self, x):
        # This is OK - BrainState handles it
        self.values.value = jnp.concatenate([self.values.value, jnp.array([x])])
        return x * 2

print("✅ Avoid modifying global variables")
print("✅ Use BrainState states for mutable data")
print("✅ Keep functions pure when possible")

### Tip 3: Batch Operations

In [ ]:
# ❌ BAD: Multiple calls
@bst.transform.jit
def process_single(x):
    return jnp.sin(x) + jnp.cos(x)

# ✅ GOOD: Batch processing
@bst.transform.jit
def process_batch(X):
    return jnp.sin(X) + jnp.cos(X)

# Generate data
data = [bst.random.randn(100) for _ in range(100)]
data_batch = jnp.stack(data)

# Multiple calls
start = time.time()
results = [process_single(x) for x in data]
time_single = (time.time() - start) * 1000

# Batch call
_ = process_batch(data_batch)  # Warmup
start = time.time()
result_batch = process_batch(data_batch)
time_batch = (time.time() - start) * 1000

print(f"Multiple calls: {time_single:.2f} ms")
print(f"Batch call: {time_batch:.2f} ms")
print(f"Speedup: {time_single / time_batch:.1f}x")
print("\n✅ Batch operations for maximum performance")

## 6. Debugging JIT Code

### Disabling JIT for Debugging

In [ ]:
@bst.transform.jit
def complex_function(x):
    # Intermediate values are traced, can't be printed during JIT
    y = jnp.sin(x)
    z = jnp.cos(y)
    return z

# To debug, temporarily disable JIT
import jax

print("With JIT (can't see intermediate values):")
result = complex_function(jnp.array(1.0))
print(f"  Result: {result}")

print("\nWith JIT disabled (for debugging):")
with jax.disable_jit():
    x = jnp.array(1.0)
    # Now we can add print statements
    y = jnp.sin(x)
    print(f"  Intermediate y: {y}")
    z = jnp.cos(y)
    print(f"  Intermediate z: {z}")

print("\n💡 Use jax.disable_jit() context manager for debugging")

## Summary

In this tutorial, you learned:

✅ **JIT Basics**
  - Using `@bst.transform.jit` decorator
  - Compilation vs execution time
  - Caching compiled functions

✅ **Static vs Dynamic Arguments**
  - Dynamic args (values) don't trigger recompilation
  - Static args (shapes, flags) create separate compilations
  - Using `static_argnums` parameter

✅ **Stateful JIT**
  - JIT with BrainState objects
  - State updates inside compiled functions
  - Neural networks with JIT

✅ **Performance**
  - 10-100x speedups typical
  - Benchmarking techniques
  - Understanding overhead

✅ **Best Practices**
  - Use JAX operations, not Python loops
  - Avoid side effects (or use States)
  - Batch operations when possible
  - Debug with `jax.disable_jit()`

### Quick Reference

| Concept | Code | When to Use |
|---------|------|-------------|
| **Basic JIT** | `@bst.transform.jit` | Most functions |
| **Static args** | `@bst.transform.jit(static_argnums=(1,))` | Loop bounds, flags |
| **Disable JIT** | `with jax.disable_jit():` | Debugging |

### Common Pitfalls

❌ **Don't:** Use Python loops over arrays  
✅ **Do:** Use JAX operations (jnp.sum, jnp.mean, etc.)

❌ **Don't:** Modify global variables  
✅ **Do:** Use BrainState states or return values

❌ **Don't:** Call JIT function in a loop with single items  
✅ **Do:** Batch the data and call once

### Performance Tips

1. 🎯 **JIT everything** that runs repeatedly
2. 📦 **Batch operations** for maximum throughput
3. 🔧 **Use static args** sparingly (each value = new compilation)
4. 💾 **Warmup** before benchmarking
5. 🧪 **Profile** to find bottlenecks

### Next Steps

Continue with:
- **Automatic Differentiation** - Computing gradients with JIT
- **Vectorization** - vmap for automatic batching
- **Advanced Transforms** - Combining JIT with other transforms